<a href="https://colab.research.google.com/github/envirodatascience/final-project-go-squirrels/blob/main/Sophie's_Squirrel_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import packages

import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
import plotnine
from plotnine import *
import geopandas as gpd

# Read in data

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Import data

In [2]:
# Read squirrel data
df_sq_2018 = pd.read_csv('/content/drive/My Drive/Squirrel/2018_Central_Park_Squirrel_Census_-_Squirrel_Data_20250409.csv')
df_sq_2018.head()

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,10142018,3,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9561344937861 40.7940823884086)
1,-73.968857,40.783783,21B-AM-1019-04,21B,AM,10192018,4,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9688574691102 40.7837825208444)
2,-73.974281,40.775534,11B-PM-1014-08,11B,PM,10142018,8,NaN,Gray,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.97428114848522 40.775533619083)
3,-73.959641,40.790313,32E-PM-1017-14,32E,PM,10172018,14,Adult,Gray,NaN,...,False,False,False,False,False,False,False,True,NaN,POINT (-73.9596413903948 40.7903128889029)
4,-73.970268,40.776213,13E-AM-1017-05,13E,AM,10172018,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9702676472613 40.7762126854894)


In [3]:
df_sq_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3023 entries, 0 to 3022
Data columns (total 31 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   X                                           3023 non-null   float64
 1   Y                                           3023 non-null   float64
 2   Unique Squirrel ID                          3023 non-null   object 
 3   Hectare                                     3023 non-null   object 
 4   Shift                                       3023 non-null   object 
 5   Date                                        3023 non-null   int64  
 6   Hectare Squirrel Number                     3023 non-null   int64  
 7   Age                                         2902 non-null   object 
 8   Primary Fur Color                           2968 non-null   object 
 9   Highlight Fur Color                         1937 non-null   object 
 10  Combination 

In [4]:
df_weather = pd.read_csv('/content/drive/My Drive/Squirrel/Oct2018_and_Mar2020_weather.csv')
df_weather.head()

,Date,Temperature,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,HDD,CDD,Precipitation,New Snow,Snow Depth
0,NaN,Maximum,Minimum,Average,Departure,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-10-01,76,63,69.5,6.0,NaN,0,5,0.00,0.0,0
2,2018-10-02,77,65,71.0,7.9,NaN,0,6,0.69,0.0,0
3,2018-10-03,74,63,68.5,5.8,NaN,0,4,0.00,0.0,0
4,2018-10-04,76,61,68.5,6.2,NaN,0,4,0.04,0.0,0


In [5]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           68 non-null     object 
 1   Temperature    69 non-null     object 
 2   Unnamed: 2     69 non-null     object 
 3   Unnamed: 3     69 non-null     object 
 4   Unnamed: 4     69 non-null     object 
 5   Unnamed: 5     0 non-null      float64
 6   HDD            68 non-null     object 
 7   CDD            68 non-null     object 
 8   Precipitation  68 non-null     object 
 9   New Snow       68 non-null     object 
 10  Snow Depth     68 non-null     object 
dtypes: float64(1), object(10)
memory usage: 6.1+ KB


# Clean Data

In [6]:
df_weather.rename(columns = {'Temperature':'Temp Max', 'Unnamed: 2':'Temp Min', 'Unnamed: 3':'Temp Avg', 'Unnamed: 4':'Temp Departure'}, inplace = True)
df_weather.head()

,Date,Temp Max,Temp Min,Temp Avg,Temp Departure,Unnamed: 5,HDD,CDD,Precipitation,New Snow,Snow Depth
0,NaN,Maximum,Minimum,Average,Departure,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-10-01,76,63,69.5,6.0,NaN,0,5,0.00,0.0,0
2,2018-10-02,77,65,71.0,7.9,NaN,0,6,0.69,0.0,0
3,2018-10-03,74,63,68.5,5.8,NaN,0,4,0.00,0.0,0
4,2018-10-04,76,61,68.5,6.2,NaN,0,4,0.04,0.0,0


In [7]:
df_weather = df_weather.drop('Unnamed: 5', axis=1) #This is a blank column that never contained any data
df_weather.head()

,Date,Temp Max,Temp Min,Temp Avg,Temp Departure,HDD,CDD,Precipitation,New Snow,Snow Depth
0,NaN,Maximum,Minimum,Average,Departure,NaN,NaN,NaN,NaN,NaN
1,2018-10-01,76,63,69.5,6.0,0,5,0.00,0.0,0
2,2018-10-02,77,65,71.0,7.9,0,6,0.69,0.0,0
3,2018-10-03,74,63,68.5,5.8,0,4,0.00,0.0,0
4,2018-10-04,76,61,68.5,6.2,0,4,0.04,0.0,0


In [8]:
df_weather = df_weather.drop(df_weather.index[0]) #drop row 0 because it was a multi-index column format and I adjusted column headings
df_weather.head()

,Date,Temp Max,Temp Min,Temp Avg,Temp Departure,HDD,CDD,Precipitation,New Snow,Snow Depth
1,2018-10-01,76,63,69.5,6.0,0,5,0.00,0.0,0
2,2018-10-02,77,65,71.0,7.9,0,6,0.69,0.0,0
3,2018-10-03,74,63,68.5,5.8,0,4,0.00,0.0,0
4,2018-10-04,76,61,68.5,6.2,0,4,0.04,0.0,0
5,2018-10-05,71,60,65.5,3.6,0,1,0.00,0.0,0


In [9]:
df_weather = df_weather.reset_index(drop=True) #reset index after dropping column and row

In [10]:
df_weather['Date'] = df_weather['Date'].astype(str) #converted to string so I can make the date formats match for merge

In [16]:
df_weather['Date'] = pd.to_datetime(df_weather['Date'], errors='coerce')
df_weather.head()

,Date,Temp Max,Temp Min,Temp Avg,Temp Departure,HDD,CDD,Precipitation,New Snow,Snow Depth
0,2018-10-01,76,63,69.5,6.0,0,5,0.00,0.0,0
1,2018-10-02,77,65,71.0,7.9,0,6,0.69,0.0,0
2,2018-10-03,74,63,68.5,5.8,0,4,0.00,0.0,0
3,2018-10-04,76,61,68.5,6.2,0,4,0.04,0.0,0
4,2018-10-05,71,60,65.5,3.6,0,1,0.00,0.0,0


In [12]:
df_sq_2018['Date'] = df_sq_2018['Date'].astype(str) #converted to string so I can make the date format match the weather df

In [13]:
def format_date(date_str):
    if len(date_str) == 7:  # For dates like 1062018 (single-digit month)
        return date_str[:1] + '/' + date_str[1:3] + '/' + date_str[3:]
    elif len(date_str) == 8:  # For dates like 10142018 (double-digit month)
        return date_str[:2] + '/' + date_str[2:4] + '/' + date_str[4:]
df_sq_2018['Date'] = df_sq_2018['Date'].apply(format_date)
df_sq_2018.head()

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,10/14/2018,3,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9561344937861 40.7940823884086)
1,-73.968857,40.783783,21B-AM-1019-04,21B,AM,10/19/2018,4,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9688574691102 40.7837825208444)
2,-73.974281,40.775534,11B-PM-1014-08,11B,PM,10/14/2018,8,NaN,Gray,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.97428114848522 40.775533619083)
3,-73.959641,40.790313,32E-PM-1017-14,32E,PM,10/17/2018,14,Adult,Gray,NaN,...,False,False,False,False,False,False,False,True,NaN,POINT (-73.9596413903948 40.7903128889029)
4,-73.970268,40.776213,13E-AM-1017-05,13E,AM,10/17/2018,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9702676472613 40.7762126854894)


In [14]:
# Convert the 'Date' column to datetime objects
df_sq_2018['Date'] = pd.to_datetime(df_sq_2018['Date'])
df_sq_2018.head()

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,2018-10-14,3,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9561344937861 40.7940823884086)
1,-73.968857,40.783783,21B-AM-1019-04,21B,AM,2018-10-19,4,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9688574691102 40.7837825208444)
2,-73.974281,40.775534,11B-PM-1014-08,11B,PM,2018-10-14,8,NaN,Gray,NaN,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.97428114848522 40.775533619083)
3,-73.959641,40.790313,32E-PM-1017-14,32E,PM,2018-10-17,14,Adult,Gray,NaN,...,False,False,False,False,False,False,False,True,NaN,POINT (-73.9596413903948 40.7903128889029)
4,-73.970268,40.776213,13E-AM-1017-05,13E,AM,2018-10-17,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9702676472613 40.7762126854894)


# Merge datasets

In [17]:
merged_df = pd.merge(
    df_weather,
    df_sq_2018,
    on='Date',
    how='inner') #only matching dates
merged_df.head()

,Date,Temp Max,Temp Min,Temp Avg,Temp Departure,HDD,CDD,Precipitation,New Snow,Snow Depth,...,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long
0,2018-10-06,68,63,65.5,4.0,0,1,0.00,0.0,0,...,False,False,False,True,True,False,False,False,NaN,POINT (-73.9674285955293 40.7829723919744)
1,2018-10-06,68,63,65.5,4.0,0,1,0.00,0.0,0,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9706105896967 40.7698124821507)
2,2018-10-06,68,63,65.5,4.0,0,1,0.00,0.0,0,...,False,False,False,False,False,False,False,False,NaN,POINT (-73.9738033257473 40.7711846894166)
3,2018-10-06,68,63,65.5,4.0,0,1,0.00,0.0,0,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9568802149818 40.7972399133816)
4,2018-10-06,68,63,65.5,4.0,0,1,0.00,0.0,0,...,False,False,False,False,False,False,True,False,NaN,POINT (-73.9576245992455 40.7873862510001)
